# **Project 02: LangChain RAG Project**


---



---


####  **Task:**

Create a Google Colab Notebook that integrates a function/tool-calling workflow using LangChain, Google Gemini Flash, and custom tools. Your system should:

- **Set Up Environment Variables:** Load the Google Gemini Flash API key securely.
- **Define the Custom Tool:** Implement a set of tools to perform some custom actions.
- **Create the Tool Wrapper for LangChain:** Integrate the calculator with LangChain as a callable tool.
- **Set Up the Google Gemini Flash Model:** Initialize the Gemini model with tool-calling capabilities.
- **Build the Conversational Chain:** Develop a conversational interface that maintains context and interacts with the tools.
- **Test the Custom Tools:** Demonstrate the system's ability to handle various queries.
- **Optional Enhancements:** Extend functionality with advanced operations, improved error handling, UI integration, or logging.

In [54]:
%pip install -qU  langchain-google-genai

In [3]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [55]:
from langchain.tools import tool

@tool
def calculator(expression: str) -> str:
    """
    A simple calculator tool that evaluates mathematical expressions.
    Args:
        expression (str): A mathematical expression to evaluate.
    Returns:
        str: The result of the evaluation.
    """
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"

@tool
def acceleration(velocity: int, time: int) -> float:
    """Find acceleration"""
    return velocity / time



In [56]:
# Define the tool
tools = [calculator, acceleration]

In [57]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp"
)

llm_with_tools = llm.bind_tools(tools)

In [58]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'calculator', 'args': {'expression': '3 * 12'}, 'id': '8e753e63-a971-4d8a-8b6d-5b6f7d35e62b', 'type': 'tool_call'}, {'name': 'calculator', 'args': {'expression': '11 + 49'}, 'id': 'b02c7313-4d81-4a91-9c2d-c8e0d97e5705', 'type': 'tool_call'}]


In [59]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"calculator": calculator, "acceleration": acceleration}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'calculator', 'arguments': '{"expression": "11 + 49"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8704166e-b6f9-4915-92c0-40b6f60efd47-0', tool_calls=[{'name': 'calculator', 'args': {'expression': '3 * 12'}, 'id': '8e753e63-a971-4d8a-8b6d-5b6f7d35e62b', 'type': 'tool_call'}, {'name': 'calculator', 'args': {'expression': '11 + 49'}, 'id': 'b02c7313-4d81-4a

In [60]:
# Invoke the llm
response = llm_with_tools.invoke(messages)

In [61]:
from IPython.display import Markdown

Markdown(response.content)

3 * 12 is 36, and 11 + 49 is 60.